# Usage of Sysinternals Tools
Detects the usage of Sysinternals Tools due to accepteula key being added to Registry

## Rule Content
```
- action: global
  title: Usage of Sysinternals Tools
  id: 25ffa65d-76d8-4da5-a832-3f2b0136e133
  status: experimental
  description: Detects the usage of Sysinternals Tools due to accepteula key being
    added to Registry
  references:
  - https://twitter.com/Moti_B/status/1008587936735035392
  date: 2017/08/28
  author: Markus Neis
  detection:
    condition: 1 of them
  falsepositives:
  - Legitimate use of SysInternals tools
  - Programs that use the same Registry Key
  level: low
- logsource:
    product: windows
    service: sysmon
  detection:
    selection1:
      EventID: 13
      TargetObject: '*\EulaAccepted'
- logsource:
    category: process_creation
    product: windows
  detection:
    selection2:
      CommandLine: '* -accepteula*'

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"13" AND registry_key_path.keyword:*\\EulaAccepted)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

In [ ]:
s = searchContext.query('query_string', query='process_command_line.keyword:*\ \-accepteula*')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()